In [1]:
!pip install arxiv
!pip install -U langchain-community
!pip install wikipedia
!pip install -U langchain-openai
!pip install langchain-google-genai
!pip install faiss-cpu
!pip install google-api-python-client>=2.100.0
!pip install entrezpy
!pip install pymupdf
!pip install beautifulsoup4 requests
!pip install crawl4ai
!pip install python-dotenv

DEPRECATION: Loading egg at /home/adityasj/.conda/envs/Adityaenv/lib/python3.11/site-packages/huggingface_hub-0.29.1-py3.8.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
DEPRECATION: Loading egg at /home/adityasj/.conda/envs/Adityaenv/lib/python3.11/site-packages/huggingface_hub-0.29.1-py3.8.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
DEPRECATION: Loading egg at /home/adityasj/.conda/envs/Adityaenv/lib/python3.11/site-packages/huggingface_hub-0.29.1-py3.8.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to us

In [2]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
import os
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.schema import Document
from langchain.tools import Tool
import requests
from bs4 import BeautifulSoup
import asyncio
from typing import List
from crawl4ai import AsyncWebCrawler, BrowserConfig, CrawlerRunConfig
from crawl4ai.markdown_generation_strategy import DefaultMarkdownGenerator
import requests
from xml.etree import ElementTree
import webbrowser
from dotenv import load_dotenv

In [18]:
load_dotenv()

google_api_key = os.getenv("GOOGLE_API_KEY")
google_cse_id = os.getenv("GOOGLE_CSE_ID")

# Wikipedia API for Document retrieval.!!

In [19]:
# api_wrapper=WikipediaAPIWrapper(top_k_results=3,doc_content_chars_max=200)
# wiki=WikipediaQueryRun(api_wrapper=api_wrapper)


from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

class WikipediaRetriever:
    def __init__(self, top_k_results=3, doc_content_chars_max=200):
        self.wrapper = WikipediaAPIWrapper(top_k_results=top_k_results, doc_content_chars_max=doc_content_chars_max)

    def search(self, query):
        """Fetches Wikipedia content and returns both text and URLs."""
        result = self.wrapper.run(query)  # Retrieve Wikipedia content
        urls = [f"https://en.wikipedia.org/wiki/{query.replace(' ', '_')}"]  # Construct Wikipedia URL
        return {"content": result, "urls": urls}

# Initialize Wikipedia Tool
api_wrapper = WikipediaAPIWrapper(top_k_results=3, doc_content_chars_max=200)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)

def wikipedia_with_urls(query):
    retriever = WikipediaRetriever()
    result = retriever.search(query)  # Get content + URL

    # ✅ Return formatted string so the LLM does not need to process `{url}`
    return f"**Response:** {result['content']}\n\n**Source:** [Click here]({result['urls'][0]})"

In [20]:
wiki.name

'wikipedia'

In [21]:
embeddings = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001",
    google_api_key = google_api_key,
    google_csi_id = google_csi_id
)

In [22]:
from langchain.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader

# Load and split documents
loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)

# Create FAISS VectorStore with Google Embeddings
vectordb = FAISS.from_documents(documents, embeddings)
retriever = vectordb.as_retriever()
retriever

USER_AGENT environment variable not set, consider setting it to identify your requests.


VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x1555176ef650>, search_kwargs={})

In [23]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,"langsmith_search",
                      "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!")

In [24]:
retriever_tool.name

'langsmith_search'

# Arxiv tool for document search. 

In [25]:
## Arxiv Tool
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper=ArxivAPIWrapper(top_k_results=3, doc_content_chars_max=200)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)

def arxiv_with_urls(query):
    """Fetches ArXiv papers and returns both content & URLs."""
    results = arxiv.run(query)  # Retrieve papers

    extracted_results = []
    for paper in results:
        extracted_results.append({
            "content": f"Title: {paper.title}\nSummary: {paper.summary}",
            "url": paper.entry_id  # ✅ ArXiv provides direct links in `entry_id`
        })

    return extracted_results if extracted_results else [{"content": "No papers found.", "url": "No URL available."}]

# Google scholar API for document search.

In [26]:
import os
from dotenv import load_dotenv
from langchain_community.utilities import GoogleSearchAPIWrapper
from langchain_community.tools import GoogleSearchRun
from langchain.tools import Tool

# ✅ Initialize Google Search API
google_scholar_wrapper = GoogleSearchAPIWrapper(
    google_api_key=google_api_key,  
    google_cse_id=google_cse_id
)
google_scholar = GoogleSearchRun(api_wrapper=google_scholar_wrapper)

# ✅ Function to Fetch Google Scholar Results with URLs
def google_scholar_with_urls(query):
    """Fetches Google Scholar results and returns both content & URLs."""
    results = google_scholar.run(query)  # Retrieve search results

    extracted_results = []
    for entry in results:
        extracted_results.append({
            "content": f"Title: {entry['title']}\nSnippet: {entry['snippet']}",
            "url": entry['link']  # ✅ Extracts the link to the source
        })

    return extracted_results if extracted_results else [{"content": "No results found.", "url": "No URL available."}]

/tmp/ipykernel_2037998/3922096516.py:8: LangChainDeprecationWarning: The class `GoogleSearchAPIWrapper` was deprecated in LangChain 0.0.33 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-google-community package and should be used instead. To use it run `pip install -U :class:`~langchain-google-community` and import as `from :class:`~langchain_google_community import GoogleSearchAPIWrapper``.
  google_scholar_wrapper = GoogleSearchAPIWrapper(
/tmp/ipykernel_2037998/3922096516.py:12: LangChainDeprecationWarning: The class `GoogleSearchRun` was deprecated in LangChain 0.0.33 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-google-community package and should be used instead. To use it run `pip install -U :class:`~langchain-google-community` and import as `from :class:`~langchain_google_community import GoogleSearchRun``.
  google_scholar = GoogleSearchRun(api_wrapper=google_scholar_wrapper)


# Source: Pubmed dataset

## Implementing FAISS for embedding and faster retrieval, since Pubmed documents can be of larger size.!!


In [28]:
import requests

class PubMedLoader:
    def __init__(self, query, top_k=3):
        self.query = query
        self.top_k = top_k
        self.api_key = os.getenv("PUBMED_API_KEY")
        self.base_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"

    def fetch_article_ids(self):
        """Fetches PubMed article IDs for a given query."""
        params = {
            "db": "pubmed",
            "term": self.query,
            "retmode": "json",
            "retmax": self.top_k,
            "api_key": self.api_key
        }
        response = requests.get(self.base_url, params=params).json()
        return response.get("esearchresult", {}).get("idlist", [])

    def fetch_article_abstracts(self, article_ids):
        """Fetches full abstracts for given PubMed article IDs."""
        articles = []
        for article_id in article_ids:
            url = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id={article_id}&retmode=text&rettype=abstract&api_key={self.api_key}"
            article_text = requests.get(url).text
            articles.append(f"PubMed ID: {article_id}\n\n{article_text}")
        return articles

    def load(self):
        """Fetches articles and returns them as raw text."""
        article_ids = self.fetch_article_ids()
        if not article_ids:
            return []

        return self.fetch_article_abstracts(article_ids)

In [30]:
# Initialize Google AI Embeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

# Initialize FAISS as None (will be created when data is available)
vectordb = None
retriever = None

# Text Splitter for chunking abstracts before storing
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

def retrieve_pubmed_articles(query):
    global vectordb, retriever  

    # Check FAISS first
    if retriever:
        similar_docs = retriever.get_relevant_documents(query)
        if similar_docs:
            print(" Retrieving from FAISS (Cached Results)")
            return similar_docs

    print(" No Cached Results, Calling PubMed API...")
    
    # Fetch fresh articles from PubMed API
    pubmed_loader = PubMedLoader(query, top_k=3)
    docs = pubmed_loader.load()

    if not docs:
        print(" No articles retrieved from PubMed.")
        return []

    print(f" Retrieved {len(docs)} articles from PubMed")  # Debugging line

    # Store new results in FAISS only if there are documents
    doc_objects = [Document(page_content=text) for text in docs]

    if doc_objects:
        chunked_documents = text_splitter.split_documents(doc_objects)

        # Debugging before storing in FAISS
        print(f" Storing {len(chunked_documents)} documents in FAISS")

        vectordb = FAISS.from_documents(chunked_documents, embeddings)
        retriever = vectordb.as_retriever()

        print(" New results stored in FAISS for future queries.")

    return doc_objects

In [31]:
# query = "Artificial Intelligence in Healthcare"
# results = retrieve_pubmed_articles(query)

# # Print results
# for idx, result in enumerate(results):
#     print(f"\n🔹 Result {idx+1}:\n{result.page_content[:1000]}...") 

# Educational pdfs and textbooks. 

In [32]:
# from crawl4ai import Crawler

class OpenTextbookCrawler:
    def __init__(self, base_url="https://open.umn.edu/opentextbooks", top_k=3):
        self.base_url = base_url
        self.top_k = top_k

    def fetch_textbooks(self):
        """Uses Crawl4AI to scrape Open Textbook Library for book titles & PDF links."""
        crawler = Crawler()
        response = crawler.get(self.base_url)

        books = []
        for book in response.extract_all({"title": "//h4/a", "pdf_link": "//a[contains(@href, 'download')]"}):
            title = book.get("title", "Unknown Title").strip()
            pdf_link = book.get("pdf_link", None)

            if pdf_link:
                books.append({"title": title, "pdf_link": pdf_link})

        return books[:self.top_k]  # Return top K books


In [33]:
import fitz  # PyMuPDF
import requests
import os

class PDFProcessor:
    def __init__(self, pdf_url, save_path="downloads"):
        self.pdf_url = pdf_url
        self.save_path = save_path

    def download_pdf(self):
        """Downloads the PDF from the provided URL."""
        response = requests.get(self.pdf_url)
        
        if response.status_code == 200:
            os.makedirs(self.save_path, exist_ok=True)
            pdf_path = os.path.join(self.save_path, self.pdf_url.split("/")[-1])

            with open(pdf_path, "wb") as f:
                f.write(response.content)
            
            return pdf_path
        return None

    def extract_text(self, pdf_path):
        """Extracts text from the given PDF."""
        doc = fitz.open(pdf_path)
        text = "\n".join(page.get_text() for page in doc)
        return text


In [34]:
from langchain.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.schema import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Initialize FAISS components
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

# Initialize FAISS as None (will be created dynamically)
vectordb = None
retriever = None

# Text Splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

def process_textbook(pdf_url):
    global vectordb, retriever  

    # Download and extract text
    processor = PDFProcessor(pdf_url)
    pdf_path = processor.download_pdf()

    if not pdf_path:
        print(" Failed to download PDF.")
        return []

    extracted_text = processor.extract_text(pdf_path)
    
    # Convert to LangChain Documents
    doc_objects = [Document(page_content=extracted_text)]

    if doc_objects:
        chunked_documents = text_splitter.split_documents(doc_objects)

        # Initialize FAISS only when we have data
        vectordb = FAISS.from_documents(chunked_documents, embeddings)
        retriever = vectordb.as_retriever()

        print(" Textbook stored in FAISS for future retrieval.")

    return doc_objects

In [35]:
from langchain.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.schema import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Initialize FAISS components
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

# Initialize FAISS as None (will be created dynamically)
vectordb = None
retriever = None

# Text Splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

def process_textbook(pdf_url):
    global vectordb, retriever  

    # Download and extract text
    processor = PDFProcessor(pdf_url)
    pdf_path = processor.download_pdf()

    if not pdf_path:
        print(" Failed to download PDF.")
        return []

    extracted_text = processor.extract_text(pdf_path)
    
    # Convert to LangChain Documents
    doc_objects = [Document(page_content=extracted_text)]

    if doc_objects:
        chunked_documents = text_splitter.split_documents(doc_objects)

        # Initialize FAISS only when we have data
        vectordb = FAISS.from_documents(chunked_documents, embeddings)
        retriever = vectordb.as_retriever()

        print(" Textbook stored in FAISS for future retrieval.")

    return doc_objects


In [36]:
from langchain.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.schema import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Initialize FAISS components
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

# Initialize FAISS as None (will be created dynamically)
vectordb = None
retriever = None

# Text Splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

def process_textbook(pdf_url):
    global vectordb, retriever  

    # Download and extract text
    processor = PDFProcessor(pdf_url)
    pdf_path = processor.download_pdf()

    if not pdf_path:
        print(" Failed to download PDF.")
        return []

    extracted_text = processor.extract_text(pdf_path)
    
    # Convert to LangChain Documents
    doc_objects = [Document(page_content=extracted_text)]

    if doc_objects:
        chunked_documents = text_splitter.split_documents(doc_objects)

        # Initialize FAISS only when we have data
        vectordb = FAISS.from_documents(chunked_documents, embeddings)
        retriever = vectordb.as_retriever()

        print(" Textbook stored in FAISS for future retrieval.")

    return doc_objects


In [37]:
def retrieve_textbook(query):
    global vectordb, retriever  

    # 1️⃣ Check FAISS First
    if retriever:
        similar_docs = retriever.get_relevant_documents(query)
        if similar_docs:
            print(" Retrieving from FAISS (Cached Results)")
            return similar_docs

    print(" No Cached Results, Scraping New Textbooks...")

    # 2️⃣ Scrape books from Open Textbook Library
    scraper = OpenTextbookCrawler(top_k=3)
    books = scraper.fetch_textbooks()

    if not books:
        print(" No textbooks found.")
        return []

    # 3️⃣ Process the first book (For now, only handling 1 book at a time)
    book = books[0]
    print(f" Downloading textbook: {book['title']}")
    
    return process_textbook(book["pdf_link"])


In [38]:
# Wikipedia Tool
wikipedia_tool = Tool(
    name="Wikipedia_Search",  # ✅ Name must be valid for Gemini API
    func=wikipedia_with_urls,  # Calls the function we modified
    description="Search for Wikipedia articles on a given topic. Returns both content and source URL."
)

# ArXiv Tool
arxiv_tool = Tool(
    name="ArXiv_Research",
    func=arxiv_with_urls,  # ✅ Uses the new function
    description="Retrieve academic research papers from ArXiv. Returns both content and source URL."
)

# Google Scholar Tool
google_scholar_tool = Tool(
    name="Google_Scholar_Search",
    func=google_scholar_with_urls,  # ✅ Uses the updated function
    description="Search for academic articles and research papers from Google Scholar. Returns both content and source URL."
)

# PubMed Tool
pubmed_tool = Tool(
    name="PubMed_Search",
    func=retrieve_pubmed_articles,  # Using the PubMed retrieval function
    description="Search for academic research papers from PubMed based on a given query. Use this tool for medical and scientific topics."
)

open_textbook_tool = Tool(
    name="Open_Textbook_Library",
    func=retrieve_textbook,
    description="Retrieve educational textbooks from Open Textbook Library. Use this tool for academic references."
)



In [39]:
tools=[wikipedia_tool,arxiv_tool,google_scholar_tool, retriever_tool, pubmed_tool, open_textbook_tool]

In [40]:
tools

[Tool(name='Wikipedia_Search', description='Search for Wikipedia articles on a given topic. Returns both content and source URL.', func=<function wikipedia_with_urls at 0x1555175ae5c0>),
 Tool(name='ArXiv_Research', description='Retrieve academic research papers from ArXiv. Returns both content and source URL.', func=<function arxiv_with_urls at 0x15552aeed800>),
 Tool(name='Google_Scholar_Search', description='Search for academic articles and research papers from Google Scholar. Returns both content and source URL.', func=<function google_scholar_with_urls at 0x1554bd55d3a0>),
 Tool(name='langsmith_search', description='Search for information about LangSmith. For any questions about LangSmith, you must use this tool!', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x15552b402b60>, retriever=VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores

In [41]:
# from langchain.tools import Tool

# for tool in tools:
#     if isinstance(tool, Tool):  # ✅ Only print if it's a Tool
#         print(f"🔹 Tool Name: {tool.name}, Function: {tool.func}")
#     else:
#         print(f"⚠️ Non-tool object found: {tool}")


In [42]:
# query = "Machine Learning"
# textbook_results = open_textbook_tool.func(query)

# # Print results
# for idx, result in enumerate(textbook_results):
#     print(f"\n🔹 Textbook Result {idx+1}:\n{result.page_content[:500]}...")

In [43]:
from dotenv import load_dotenv

load_dotenv()
import os

llm = ChatGoogleGenerativeAI(model="gemini-2.0-pro-exp-02-05", temperature=0)

In [44]:
# from langchain import hub
# # Get the prompt to use - you can modify this!
# prompt = hub.pull("hwchase17/openai-functions-agent")
# prompt.messages.insert(0, "When answering a question, always include both the content and the source URL if available.")
# prompt.messages

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, HumanMessagePromptTemplate, SystemMessagePromptTemplate

updated_prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(
        "You are a helpful assistant. When answering a question, always include both the content and the source URL if available."
    ),
    MessagesPlaceholder(variable_name="chat_history", optional=True),
    HumanMessagePromptTemplate.from_template("{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

updated_prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant. When answering a question, always include both the content and the source URL if available.'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [45]:
### Agents
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,updated_prompt)

In [46]:
## Agent Executer
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [47]:
query = "Quantum Computing"
agent_response = agent_executor.invoke({"input": query})

# Extract content and URL from agent response
content = agent_response.get("output", "No content available.")
url = agent_response.get("url", "No URL available.")  # Ensure it handles missing URLs gracefully

# Print formatted response
print(f"\n Content:\n{content}")
print(f" Source: {url}")



> Entering new AgentExecutor chain...

Invoking: `Wikipedia_Search` with `{'query': 'Quantum Computing'}`


**Response:** Page: Quantum computing
Summary: A quantum computer is a computer that exploits quantum mechanical phenomena. On small scales, physical matter exhibits properties of both particles and waves, and quan

**Source:** [Click here](https://en.wikipedia.org/wiki/Quantum_Computing)Quantum computing is a type of computation that harnesses the principles of quantum mechanics. Unlike classical computers that store information as bits representing 0 or 1, quantum computers use quantum bits, or qubits. Qubits can represent 0, 1, or a combination of both states simultaneously through a concept called superposition. This, along with other quantum phenomena like entanglement and interference, allows quantum computers to perform certain calculations much faster than classical computers.

**Source:** [https://en.wikipedia.org/wiki/Quantum_Computing](https://en.wikipedia.org/wiki/Q

In [72]:
webbrowser.open(url)

False

In [79]:
query = "Quantum Computing"
wiki_results = wikipedia_tool.func(query)

# ✅ Extract content and first URL
content = wiki_results.get("content", "No content available.")
url = wiki_results.get("urls", ["No URL available."])[0]  # ✅ Ensure we get the first URL

# ✅ Print formatted response
print(f"\n Wikipedia Content:\n{content[:500]}...")  # ✅ Truncate content for readability
print(f"🔗 Wikipedia Source: {url}")  # ✅ Ensure proper URL extraction

AttributeError: 'str' object has no attribute 'get'